<a href="https://colab.research.google.com/github/tnfru/PPO/blob/master/Cartpole.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions.categorical import Categorical
from collections import deque
from torch.utils.data import DataLoader
from tqdm.notebook import trange
import wandb

In [3]:
gym.envs.register(
    id='CartPole-v2000',
    entry_point='gym.envs.classic_control:CartPoleEnv',
    max_episode_steps=2000
)

In [19]:
def normalize(x):
    if torch.isnan(x.std()):
        return x - x.mean(0)
    
    return (x - x.mean(0)) / (x.std(0) + 1e-8)

In [92]:
class Trajectory(torch.utils.data.Dataset):
    def __init__(self):
        self.states = []
        self.log_probs = []
        self.actions = []
        self.expected_returns = []
        self.dones = []
        self.advantages = []

    def __len__(self):
        return len(self.states)

    def append_trajectory(self, states, actions, expected_returns, dones, log_probs, advantages):
        self.states.extend(states)
        self.actions.extend(actions)
        self.expected_returns.extend(expected_returns)
        self.dones.extend(dones)
        self.log_probs.extend(log_probs)
        self.advantages.extend(advantages)
    
    def convert_to_advantages(self, rewards, state_vals, discount_factor, gae_lambda=0.95):
        advantages = []
        advantage = 0
        next_state_value = 0

        for reward, state_val in zip(reversed(rewards), reversed(state_vals)):
            td_error = reward + discount_factor * next_state_value - state_val
            advantage = td_error + discount_factor * gae_lambda * advantage
            next_state_value = state_val
            advantages.insert(0, advantage)

        return torch.tensor(advantages, dtype=torch.float)
        
    def fix_datatypes(self):
        self.states = torch.stack(self.states)
        self.actions = torch.tensor(self.actions, dtype=torch.long)
        self.dones = torch.tensor(self.dones, dtype=torch.int)
        self.log_probs = torch.tensor(self.log_probs, dtype=torch.float)
        

    def clear_memory(self):
        self.states = []
        self.log_probs = []
        self.actions = []
        self.rewards = []
        self.dones = []
    
    def __getitem__(self, index):
        state = self.states[index]
        action = self.actions[index]
        expected_return = self.expected_returns[index]
        done = self.dones[index]
        log_prob = self.log_probs[index]
        advantage = self.advantages[index]
        
        return state, action, expected_return, done, log_prob, advantage

In [106]:
class Agent:
    def __init__(self, env, action_dim, state_dim, batch_size, clip_ratio=0.25):
        self.env = env
        self.actor = Policy(action_dim, state_dim)
        self.critic = CriticNet(state_dim)
        self.trajectories = deque(maxlen=2)
        self.discount_factor = 0.99
        self.batch_size = batch_size
        self.clip_ratio = clip_ratio
        self.actor_opt = optim.Adam(self.actor.parameters(), lr=3e-4)
        self.critic_opt = optim.Adam(self.critic.parameters(), lr=1e-3)
        self.critic_loss = nn.MSELoss()
        self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        self.train_iterations = 1
        
        # initialize logging
        wandb.init(project="cartpole")
        wandb.watch(self.actor, log="all")
        wandb.watch(self.critic, log="all")
        wandb.run.name = 'gae_lamba_' + str(self.train_iterations) + '_' + wandb.run.name 
        
    def get_action(self, obs):
        action_probs = self.actor(obs)

        action_dist = Categorical(logits=action_probs)
        action = action_dist.sample()
        state_val = self.critic(obs)
        
        return action.item(), action_dist.log_prob(action).item(), state_val.squeeze().item()
        
    
    def train(self):
        for i in range(self.train_iterations):
            loader = DataLoader(self.trajectory, batch_size=self.batch_size, shuffle=True, pin_memory=True)
                             #  drop_last=True)
            
            for states, actions, expected_returns, dones, old_log_probs, advantages in loader:
                states = states.to(self.device)
                actions = actions.to(self.device)
                exptected_returns = expected_returns.to(self.device)
                advantages = advantages.to(self.device)
                old_log_probs = old_log_probs.to(self.device)
                
                action_probs = self.actor(states)
                action_dist = Categorical(logits=action_probs)
                log_probs = action_dist.log_prob(actions)
                
                # log trick for efficient computational graph during backprop
                ratio = torch.exp(log_probs - old_log_probs)
                
                weighted_objective = advantages * ratio
                clipped_objective = torch.clamp(ratio, 1 - self.clip_ratio, 1 + self.clip_ratio) * advantages
                
                actor_loss = -torch.min(weighted_objective, clipped_objective).mean()
                
                state_values = self.critic(states)
                critic_loss = self.critic_loss(state_values, exptected_returns.unsqueeze(1))
                
                self.actor_opt.zero_grad()
                actor_loss.backward(retain_graph=True)
                self.actor_opt.step()
                
                self.critic_opt.zero_grad()
                critic_loss.backward()
                self.critic_opt.step()
                
                wandb.log({'critic loss': critic_loss.mean()})
                wandb.log({'avg state value': state_values.mean()})
        
    
    def run_episode(self, trajectory=None, render=False):
        observations = []
        actions = []
        rewards = []
        dones = []
        log_probs = []
        state_vals = []

        done = False
        obs = self.env.reset()
        
        while not done:
            if render:
                self.env.render()

            obs = torch.tensor(obs, dtype=torch.float, device=self.device)
            action, log_prob, state_val = agent.get_action(obs)
            obs = obs.cpu()
            next_obs, reward, done, _ = self.env.step(action)
            
            observations.append(obs)
            state_vals.append(state_val)
            actions.append(action)
            rewards.append(reward)
            dones.append(done)
            log_probs.append(log_prob)

            obs = next_obs
            
        wandb.log({'reward': np.sum(rewards)})
        
        if trajectory is not None:
            advantages = trajectory.convert_to_advantages(rewards, state_vals, self.discount_factor)
            expected_returns = torch.tensor(state_vals, dtype=torch.float) + advantages
            advantages = normalize(advantages)
            trajectory.append_trajectory(observations, actions, expected_returns, dones, log_probs, advantages)
            return trajectory


    def run_timesteps(self, num_timesteps):
        timesteps = 0
        trajectory = Trajectory()

        while len(trajectory) < num_timesteps:
            trajectory = self.run_episode(trajectory)
        trajectory.fix_datatypes()
        self.trajectory = trajectory


In [107]:
class Policy(nn.Module):
    def __init__(self, action_dim, state_dim):
        super(Policy, self).__init__()
        self.fc1 = nn.Sequential(
            nn.Linear(state_dim, 64),
            nn.ReLU()
        )
        
        self.fc2 = nn.Sequential(
            nn.Linear(64, 64),
            nn.ReLU()
        )
        
        self.fc3 = nn.Sequential(
            nn.Linear(64, action_dim)
        )
        
        self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        self.to(self.device)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        
        return x

In [108]:
class CriticNet(nn.Module):
    def __init__(self, state_dim):
        super(CriticNet, self).__init__()
        self.fc1 = nn.Sequential(
            nn.Linear(state_dim, 64),
            nn.ReLU()
        )
        
        self.fc2 = nn.Sequential(
            nn.Linear(64, 64),
            nn.ReLU()
        )
        
        self.fc3 = nn.Sequential(
            nn.Linear(64, 1)
        )       
        
        self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        self.to(self.device)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        
        return x

In [ ]:
env = gym.make("CartPole-v2000")
agent = Agent(env, 2, 4, 32)

for i in range(1000):
    agent.run_timesteps(500)
    agent.train()
        

reward,2000.0
_runtime,177
_timestamp,1627770493
_step,5446
critic loss,1.28629
avg state value,92.97242


reward,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▃▃▅█
_runtime,▁▁▁▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇███
_timestamp,▁▁▁▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇███
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
critic loss,▄▃▃▃▄▄▃▄▅▃▄▆▄▄▃█▄▃▂▄▅▇▁▂▃▂▂▁▁▁▅▃▁▃▁▁▁▃▁▁
avg state value,▁▁▂▂▃▃▃▃▄▅▅▅▅▇▇▇▆▆▇▇▇▇▇▇█▇█▇▇███▇███████
